In [10]:
from synapse.ml.train import TrainClassifier
from pyspark.ml import Pipeline
from synapse.ml import *
from synapse.ml.lightgbm import LightGBMClassifier
from synapse.ml.automl import *
from pyspark.sql.functions import col,when,regexp_replace,rand
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder, TrainValidationSplit
from pyspark.ml.feature import VectorAssembler
from pyspark.sql import SparkSession
import numpy as np
from pyspark.sql.types import IntegerType

In [9]:
spark = SparkSession.builder.appName("BayesianOptimization").getOrCreate()

spark = SparkSession.builder.master('yarn').appName("GridSearch").getOrCreate()
df = spark.read.csv('/dataset/final_all_onehotencoding_v3.csv', header=True, inferSchema=True)

for col_name in df.columns:
    df = df.withColumn(col_name, when(
        col(col_name).isNull(), 0.0).otherwise(col(col_name)))

all_columns = df.columns
columns_to_convert = all_columns
for column in columns_to_convert:
    df = df.withColumn(column, col(column).cast(IntegerType()))

# Train, val, test : 0.9, 0.05, 0.05
train_ratio = 0.9
val_ratio = 0.05
test_ratio = 0.05

train_data, val_data, test_data = df.randomSplit(
    [train_ratio, val_ratio, test_ratio], seed=37)

feature_columns = df.columns
feature_columns.remove("TARGET")

# combine features into a single vector column
assembler = VectorAssembler(inputCols=feature_columns, outputCol="features")
train = assembler.transform(train_data)
val = assembler.transform(val_data)
test = assembler.transform(test_data)

In [13]:
gbt = LightGBMClassifier(
    objective='binary',
    featuresCol="features",
    labelCol='TARGET',
    isUnbalance=True,
    metric='auc',
)



smlmodels = [gbt]
mmlmodels = [TrainClassifier(model=model, labelCol="TARGET") for model in smlmodels]



paramBuilder =(
     HyperparamBuilder()\
    .addHyperparam(gbt,gbt.maxDepth,DiscreteHyperParam([6,7]))\
    .addHyperparam(gbt,gbt.maxBin,RangeHyperParam(8,16))\
)

searchSpace = paramBuilder.build()
print(f'seachSpace:{searchSpace}')
randomSpace = RandomSpace(searchSpace)
print(f'randomSpace:{randomSpace}')

'''
param_grid = ParamGridBuilder() \
    .addGrid(gbt.maxDepth, np.arange(6,7)) \
    .addGrid(gbt.maxBin, [8, 16]) \
    .build()

# '''


# print(" HyperparamBuilder Success")

# evaluator = BinaryClassificationEvaluator(
#     labelCol="TARGET", rawPredictionCol="prediction", metricName='areaUnderROC')

# # Run cv + trainvalsplit, and choose the best set of parameters.
# cv = CrossValidator(estimator=gbt,
#                     estimatorParamMaps=param_grid,
#                     evaluator=evaluator,
#                     numFolds=3)

# cv_model = cv.fit(train)
# model = cv_model.bestModel

# # Make predictions
# cv_predictions = cv_model.transform(test)

# auc = evaluator.evaluate(cv_predictions)
# print(f'CV AUC: {auc:.4f}')


# java_model = model.stages[-1]._java_obj
# for param in paramGrid[0]:
#     param_name = param.name
#     param_value = java_model.getOrDefault(java_model.getParam(param_name))
#     print(f"{param_name}: {param_value}")

TypeError: 'JavaPackage' object is not callable